In [1]:
!pip install accelerate peft trl datasets bitsandbytes auto-gptq optimum -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,GPTQConfig
from peft import LoraConfig,prepare_model_for_kbit_training, get_peft_model

model_ckpt = "Neupane9Sujal/llama-gptq-medical-finetuned-chatbot"
tokenizer = AutoTokenizer.from_pretrained(
    model_ckpt
)

quantization_config = GPTQConfig(bits=4,disable_exllama=True,tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained(
    model_ckpt,
    revision='main',
    quantization_config=quantization_config,
    device_map='auto')

tokenizer_config.json:   0%|          | 0.00/820 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


...

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

In [4]:
import time

while True:
    prompt_template = "[INST] You are a medical chatbot. Your job is to give proper instructions to the question that is being asked. If you don't know the answer, don't make something up. [/INST]"
    user_input = input("User: ")
    prompt = f"{prompt_template} {user_input}"

    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    start_time = time.time()
    with torch.no_grad():
        output = tokenizer.decode(model.generate(**model_input, max_new_tokens=100, repetition_penalty=1.15)[0], skip_special_tokens=True)
    end_time = time.time()

    inference_time = end_time - start_time

    print("ChatBot:", output)
    print("Inference Time:", inference_time)


User: Why should a person be advised to eat healthy diet?
ChatBot: [INST] You are a medical chatbot. Your job is to give proper instructions to the question that is being asked. If you don't know the answer, don't make something up. [/INST] Why should a person be advised to eat healthy diet?  Because it will help him in maintaining his weight and also prevent any disease like obesity or heart diseases. So he can take more fruits and vegetables in his diet. He can avoid junk foods as they contain high calories which may lead to obesity. Also, he can reduce sugar intake because it contains high calories too. Hope I have answered your query. Let me know if I can assist you further.</s>
Inference Time: 75.13251876831055
User: Is it a healthy habit to exercise regularly?
ChatBot: [INST] You are a medical chatbot. Your job is to give proper instructions to the question that is being asked. If you don't know the answer, don't make something up. [/INST] Is it a healthy habit to exercise regula

KeyboardInterrupt: ignored